In [1]:
import pandas as pd
from pymongo import MongoClient
from pprint import pprint
from dotenv import load_dotenv
from binance.client import Client
from binance import AsyncClient
import glob
import os
from pathlib import Path
import asyncio

In [2]:
# Load Environment Variables
load_dotenv()

# Gets MongoDB Connection String
MDB_CONNECTION_STRING = os.getenv('MDB_CONNECTION_STRING')

# Gets Binance Api Key and Api Secret
BINANCE_API_KEY = os.getenv('BINANCE_API_KEY')
BINANCE_API_SECRET = os.getenv('BINANCE_API_SECRET')

In [3]:
# Function to connect to the Mongo DB
def get_database():
    try:
        client = MongoClient(MDB_CONNECTION_STRING)
        db = client["project-02"]
        return db
    except Exception as e:
        print(e)

In [4]:
# Connect to the db
db = get_database()

In [5]:
# Test Connection
serverStatusResult=db.command("serverStatus")
print(serverStatusResult["version"])

5.0.6


In [6]:
# Get the coinpairs from the Database
db_coinpairs = db["coinpairs"].find({"exchange" : "binance"})

# convert the dictionary objects to dataframe
binance_coinpairs_df = pd.DataFrame(db_coinpairs)

# see the magic
coinpair_list = list(binance_coinpairs_df['pair'])
print(coinpair_list)

['ADABUSD', 'SOLBUSD', 'ETHBUSD', 'BTCBUSD']


In [7]:
# Get the timeframes from the Database
db_binance_timeframes = db["binance_timeframes"].find()

# convert the dictionary objects to dataframe
binance_timeframes_df = pd.DataFrame(db_binance_timeframes)

# see the magic
binance_timeframes_list = list(binance_timeframes_df['timeframe'])
print(binance_timeframes_list)

['1m', '5m', '30m', '1h', '1d']


In [8]:
# Get latest inserted kline date for each Coin Pair
def get_pair_data(binance_timeframes_list, coinpair_list):
    complete_pair_tf = []
    from_timestamp = "3 months ago UTC"
    for timeframe in binance_timeframes_list:
        for pair in coinpair_list:
            # Get the coinpairs from the Database
            collection_name = pair+"_"+timeframe
            latest = db[collection_name].find().sort('open_time', -1 ).limit(1)
            # Exists, update collection by setting from_timestamp to lastest timestamp
            if collection_name in db.list_collection_names():
                complete_pair_tf.append([timeframe, pair, latest[0]["open_time"].strftime("%m/%d/%Y, %H:%M:%S"),collection_name, "is_update"])
            # Does not exists, import from csv
            else:
                complete_pair_tf.append([timeframe, pair, from_timestamp, collection_name, "is_new"])
    return complete_pair_tf

In [9]:
# Function to get the kline data from Binance
async def get_coinpair_kline(pair, timeframe, from_timestamp):
    client = await AsyncClient.create(BINANCE_API_KEY, BINANCE_API_SECRET)
    if timeframe == "1m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1MINUTE, from_timestamp)
    elif timeframe == "5m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_5MINUTE, from_timestamp)
    elif timeframe == "30m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_30MINUTE, from_timestamp)
    elif timeframe == "1h":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1HOUR, from_timestamp)
    elif timeframe == "1d":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1DAY, from_timestamp)
    else:
        return
    await client.close_connection()
    return klines

In [10]:
async def get_binance_data(complete_pair_tf):
    headers=["open_time", "open", "high", "low","close","volume","close_time","quote_asset_volume","numer_trades","taker_base_volume","taker_quote_volume","ignore"]
    for query_pair in complete_pair_tf:
        collection_name = query_pair[1]+"_"+query_pair[0]
        print(f"Processing {collection_name}..")
        kline_list = await get_coinpair_kline(query_pair[1], query_pair[0], query_pair[2])
        # If it is updating the db then delete the first item as it repeats
        if query_pair[4] == "is_update":
            kline_list.pop(0)
            print(f"Updating {len(kline_list)} items..")
        else:
            print(f"Adding {len(kline_list)} items..")
        if len(kline_list) > 0:
            kline_df = pd.DataFrame(kline_list, columns=headers)
            kline_df['open_time'] = kline_df['open_time'].values.astype(dtype='datetime64[ms]')
            kline_df['close_time'] = kline_df['close_time'].values.astype(dtype='datetime64[ms]')
            kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]] = kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]].astype(float)
            kline_df = kline_df.drop(columns=['ignore'])
            kline_dict = kline_df.to_dict("records")
            #print(kline_dict[0])
            db[query_pair[3]].insert_many(kline_dict)
        print(f"Done :)")


In [11]:
# Get pair info to update DB
complete_pair_tf = get_pair_data(binance_timeframes_list, coinpair_list)
# print(complete_pair_tf)

[['1m', 'ADABUSD', '04/04/2022, 18:07:00', 'ADABUSD_1m', 'is_update'], ['1m', 'SOLBUSD', '04/04/2022, 18:07:00', 'SOLBUSD_1m', 'is_update'], ['1m', 'ETHBUSD', '04/04/2022, 18:07:00', 'ETHBUSD_1m', 'is_update'], ['1m', 'BTCBUSD', '04/04/2022, 18:07:00', 'BTCBUSD_1m', 'is_update'], ['5m', 'ADABUSD', '04/04/2022, 18:05:00', 'ADABUSD_5m', 'is_update'], ['5m', 'SOLBUSD', '04/04/2022, 18:05:00', 'SOLBUSD_5m', 'is_update'], ['5m', 'ETHBUSD', '04/04/2022, 18:05:00', 'ETHBUSD_5m', 'is_update'], ['5m', 'BTCBUSD', '04/04/2022, 18:05:00', 'BTCBUSD_5m', 'is_update'], ['30m', 'ADABUSD', '04/04/2022, 18:00:00', 'ADABUSD_30m', 'is_update'], ['30m', 'SOLBUSD', '04/04/2022, 18:00:00', 'SOLBUSD_30m', 'is_update'], ['30m', 'ETHBUSD', '04/04/2022, 18:00:00', 'ETHBUSD_30m', 'is_update'], ['30m', 'BTCBUSD', '04/04/2022, 18:00:00', 'BTCBUSD_30m', 'is_update'], ['1h', 'ADABUSD', '04/04/2022, 18:00:00', 'ADABUSD_1h', 'is_update'], ['1h', 'SOLBUSD', '04/04/2022, 18:00:00', 'SOLBUSD_1h', 'is_update'], ['1h', 'ETH

In [ ]:
# Update DB with the latest binance Data
await get_binance_data(complete_pair_tf)

Processing ADABUSD_1m..
Updating 2 items
Done :)
Processing SOLBUSD_1m..
Updating 2 items
Done :)
Processing ETHBUSD_1m..
Updating 2 items
Done :)
Processing BTCBUSD_1m..
Updating 2 items
Done :)
Processing ADABUSD_5m..
Updating 0 items
Done :)
Processing SOLBUSD_5m..
Updating 0 items
Done :)
Processing ETHBUSD_5m..
Updating 0 items
Done :)
Processing BTCBUSD_5m..
Updating 0 items
Done :)
Processing ADABUSD_30m..
Updating 0 items
Done :)
Processing SOLBUSD_30m..
Updating 0 items
Done :)
Processing ETHBUSD_30m..
Updating 0 items
Done :)
Processing BTCBUSD_30m..
Updating 0 items
Done :)
Processing ADABUSD_1h..
Updating 0 items
Done :)
Processing SOLBUSD_1h..
Updating 0 items
Done :)
Processing ETHBUSD_1h..
Updating 0 items
Done :)
Processing BTCBUSD_1h..
